In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Extract CSV files into DataFrames

In [2]:
climbing_file = "Resources/climbing_statistics.csv"
climbing_df = pd.read_csv(climbing_file)
climbing_df

,Date,Route,Attempted,Succeeded,Success Percentage
0,11/27/2015,Disappointment Cleaver,2,0,0.000000
1,11/21/2015,Disappointment Cleaver,3,0,0.000000
2,10/15/2015,Disappointment Cleaver,2,0,0.000000
3,10/13/2015,Little Tahoma,8,0,0.000000
4,10/9/2015,Disappointment Cleaver,2,0,0.000000
...,...,...,...,...,...
4072,1/16/2014,Little Tahoma,2,0,0.000000
4073,1/6/2014,Disappointment Cleaver,8,0,0.000000
4074,1/6/2014,Disappointment Cleaver,8,0,0.000000
4075,1/5/2014,Disappointment Cleaver,2,0,0.000000


In [3]:
weather_file = "Resources/Rainier_Weather.csv"
weather_df = pd.read_csv(weather_file)
weather_df

,Date,Battery Voltage AVG,Temperature AVG,Relative Humidity AVG,Wind Speed Daily AVG,Wind Direction AVG,Solare Radiation AVG
0,12/31/2015,13.845000,19.062917,21.870833,21.977792,62.325833,84.915292
1,12/30/2015,13.822917,14.631208,18.493833,3.540542,121.505417,86.192833
2,12/29/2015,13.834583,6.614292,34.072917,0.000000,130.291667,85.100917
3,12/28/2015,13.710417,8.687042,70.557917,0.000000,164.683750,86.241250
4,12/27/2015,13.362500,14.140417,95.754167,0.000000,268.479167,31.090708
...,...,...,...,...,...,...,...
459,9/27/2014,13.467083,33.318750,60.200417,7.140917,116.294583,227.597125
460,9/26/2014,13.532500,28.238333,100.000000,0.897625,259.645833,114.400833
461,9/25/2014,13.648333,26.823750,99.854167,3.166208,278.604167,142.698917
462,9/24/2014,13.168750,29.702917,100.000000,45.915000,278.054167,37.873708


# Transform Data

In [23]:
# Remove unwanted columns from Weather DF
new_weather_df = weather_df[['Date', 'Temperature AVG', 'Relative Humidity AVG', 'Wind Speed Daily AVG']].copy()

renamed_weather_df = new_weather_df.rename(columns={'Date':'date', 'Temperature AVG': 'temp_avg', 
    'Relative Humidity AVG':'relative_humidity_avg', 'Wind Speed Daily AVG': 'wind_speed_daily_avg'})
renamed_weather_df.head()

,date,temp_avg,relative_humidity_avg,wind_speed_daily_avg
0,12/31/2015,19.062917,21.870833,21.977792
1,12/30/2015,14.631208,18.493833,3.540542
2,12/29/2015,6.614292,34.072917,0.000000
3,12/28/2015,8.687042,70.557917,0.000000
4,12/27/2015,14.140417,95.754167,0.000000


In [25]:
# Clean climbing statistics data by dropping duplicates and setting the index
new_climbing_df = climbing_df.drop_duplicates(subset =["Date","Route"], keep='last')

renamed_climbing_df = new_climbing_df.rename(columns={'Date':'date', 'Route': 'route', 'Attempted':'attempted',
    'Succeeded': 'succeeded', 'Success Percentage':'success_percentage'})


renamed_climbing_df.head(20)

,date,route,attempted,succeeded,success_percentage
0,11/27/2015,Disappointment Cleaver,2,0,0.000000
1,11/21/2015,Disappointment Cleaver,3,0,0.000000
2,10/15/2015,Disappointment Cleaver,2,0,0.000000
3,10/13/2015,Little Tahoma,8,0,0.000000
4,10/9/2015,Disappointment Cleaver,2,0,0.000000
6,10/3/2015,Disappointment Cleaver,2,0,0.000000
7,10/2/2015,Kautz Glacier,2,0,0.000000
8,10/2/2015,Disappointment Cleaver,2,0,0.000000
9,9/30/2015,Disappointment Cleaver,2,0,0.000000
10,9/28/2015,Disappointment Cleaver,12,4,0.333333


In [43]:
# Rename a couple of routes to reconcile with routes table below
renamed_climbing_df = renamed_climbing_df.replace({'route' : { 'Kautz Glacier' : 'Kautz Glacier Direct',
    'Emmons-Winthrop' : 'Emmons Glacier', 'Fuhrers Finger' : 'Fuhrer Finger' }})

renamed_climbing_df.head(20)

,date,route,attempted,succeeded,success_percentage
0,11/27/2015,Disappointment Cleaver,2,0,0.000000
1,11/21/2015,Disappointment Cleaver,3,0,0.000000
2,10/15/2015,Disappointment Cleaver,2,0,0.000000
3,10/13/2015,Little Tahoma,8,0,0.000000
4,10/9/2015,Disappointment Cleaver,2,0,0.000000
6,10/3/2015,Disappointment Cleaver,2,0,0.000000
7,10/2/2015,Kautz Glacier Direct,2,0,0.000000
8,10/2/2015,Disappointment Cleaver,2,0,0.000000
9,9/30/2015,Disappointment Cleaver,2,0,0.000000
10,9/28/2015,Disappointment Cleaver,12,4,0.333333


## Connect to local database

In [20]:
connection_string = "postgres:postgres@localhost:5432/Mt_Rainier_db"
engine = create_engine(f'postgresql://{connection_string}')

## Check for tables

In [21]:
engine.table_names()

['weather', 'climbing_statistics']

## Load DataFrames into SQL database

In [44]:
renamed_climbing_df.to_sql(name='climbing_statistics', con=engine, if_exists='append', index=False)

In [27]:
renamed_weather_df.to_sql(name='weather', con=engine, if_exists='append', index=False)

## Scrape Table to add to DB

In [30]:
url = 'https://www.summitpost.org/mount-rainier/150291#chapter_6'

In [31]:
tables = pd.read_html(url)
tables

[            0                        1
 0  Page Type:            Mountain/Rock
 1    Lat/Lon:   46.85280°N / 121.759°W
 2  Elevation:        14411 ft / 4392 m
 3         NaN  Sign the Climber's Log!,
                    0                     1
 0           Location  Max Number of People
 1     Muir Snowfield                    36
 2          Camp Muir                   110
 3     Ingraham Flats                    35
 4        Camp Hazard                    36
 5  Kautz Alpine Zone                    36
 6      Camp Schurman                    48
 7       Emmons Flats                    24
 8         Thumb Rock                    12
 9     Rainier Summit                    36,
                       Route Name  Duration Difficulty (Alpine Grade)  \
 0         Disappointment Cleaver    2 days                        II   
 1        Ingraham Glacier Direct    2 days                        II   
 2       Ingraham, Gibraltar Rock    2 days                        II   
 3                 Gib

In [35]:
# Select just the first table
route_table = tables[2]
route_table

,Route Name,Duration,Difficulty (Alpine Grade),Elevation Gain,Max Grade,SeasonApproach,Unnamed: 6
0,Disappointment Cleaver,2 days,II,"9,000 feet",35 degrees,June to Sept.,Paradise
1,Ingraham Glacier Direct,2 days,II,"9,000 feet",35 degrees,January to May,Paradise
2,"Ingraham, Gibraltar Rock",2 days,II,"9,000 feet",50 degrees,January to May,Paradise
3,Gibraltar Rock,2-3 days,V,"9,000 feet",80 degrees,January to May,Paradise
4,Gibraltar Ledges,2 days,II,"9,000 feet",50 degrees,January to May,Paradise
...,...,...,...,...,...,...,...
56,Russell Cliff Gully,2 days,II+,"10,000 feet",50 degrees,January to July,White River
57,Russell Cliff Central Bowl,2 days,II+,"10,000 feet",50 degrees,January to July,White River
58,Russell Cliff Upper Headwall,2 days,II+,"10,000 feet",50 degrees,January to July,White River
59,Winthrop Glacier,2 days,II,"10,000 feet",40 degrees,January to Sept.,White River


In [36]:
route_table = route_table[['Route Name', 'Difficulty (Alpine Grade)', 'Elevation Gain', 'Max Grade', 'SeasonApproach']]

In [37]:
route_table

,Route Name,Difficulty (Alpine Grade),Elevation Gain,Max Grade,SeasonApproach
0,Disappointment Cleaver,II,"9,000 feet",35 degrees,June to Sept.
1,Ingraham Glacier Direct,II,"9,000 feet",35 degrees,January to May
2,"Ingraham, Gibraltar Rock",II,"9,000 feet",50 degrees,January to May
3,Gibraltar Rock,V,"9,000 feet",80 degrees,January to May
4,Gibraltar Ledges,II,"9,000 feet",50 degrees,January to May
...,...,...,...,...,...
56,Russell Cliff Gully,II+,"10,000 feet",50 degrees,January to July
57,Russell Cliff Central Bowl,II+,"10,000 feet",50 degrees,January to July
58,Russell Cliff Upper Headwall,II+,"10,000 feet",50 degrees,January to July
59,Winthrop Glacier,II,"10,000 feet",40 degrees,January to Sept.


In [38]:
# Rename columns
routes = route_table.rename(columns={'Route Name':'route_name', 'Difficulty (Alpine Grade)': 'difficulty_rating', 
    'Elevation Gain': 'elevation_gain', 'Max Grade': 'max_grade', 'SeasonApproach':'season_approach'})
routes

,route_name,difficulty_rating,elevation_gain,max_grade,season_approach
0,Disappointment Cleaver,II,"9,000 feet",35 degrees,June to Sept.
1,Ingraham Glacier Direct,II,"9,000 feet",35 degrees,January to May
2,"Ingraham, Gibraltar Rock",II,"9,000 feet",50 degrees,January to May
3,Gibraltar Rock,V,"9,000 feet",80 degrees,January to May
4,Gibraltar Ledges,II,"9,000 feet",50 degrees,January to May
...,...,...,...,...,...
56,Russell Cliff Gully,II+,"10,000 feet",50 degrees,January to July
57,Russell Cliff Central Bowl,II+,"10,000 feet",50 degrees,January to July
58,Russell Cliff Upper Headwall,II+,"10,000 feet",50 degrees,January to July
59,Winthrop Glacier,II,"10,000 feet",40 degrees,January to Sept.


In [40]:
routes.to_sql(name='routes', con=engine, if_exists='append', index=False)